In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
import xclim as xclim

In [ ]:
xclim.core.options.set_options(data_validation='log') # 关闭CF文件规范的检查
xclim.set_options(check_missing="pct", missing_options={"pct": {"tolerance": 1.}}) # 计算一年中的片段（例如4-8月），关闭NAN的使用

# XCLIM指标计算
2021.12.30

renew:
- 2021.01.05
    
    暂时不考虑百分比指数的绘制

## 降水部分

### 数据读入
日降水数据，修改坐标名称

In [ ]:
dir_in          = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/ordata/"
filename_obs    = "obsmerge_pre_98-17.nc"
filename_vr     = "vr_pre_98-17.nc"
filename_rcm    = "rcm_pre_98-17.nc"

ds_or = {}
ds_or['obs']    = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['obs']      = ds_or['obs']['premerge']#[:,  :, :]
var['vr']       = ds_or['vr']['precip_MPAS']
var['rcm']      = ds_or['rcm']['precip_MPAS']

var['obs'] = var['obs'].reset_coords(names = 'lev', drop = True) # 去除掉obs中多余的lev coords

# change coords
var_list = ['obs', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var[i].coords.keys(), var['obs'].coords.keys()))
#     # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var[i] = var[i].rename(rename_dict)
    var[i]._coords = var['obs']._coords
    var[i] = var[i].rename(i)

### 极端指数的计算

In [ ]:
def calc_extreme_pre( var_temp, fillna = -1e30 ):
    """
    输入一个降水的DataArray，使用xlim计算下面的极端气候指数，freq = 'MS'
    """
    import xclim as xclim
    xclim.core.options.set_options(data_validation='log')
    
    var_temp = var_temp.rename("pr")
    var_temp.attrs['units'] = "mm/d"

    dict_out_temp = {}

    # R95p
    # obs_temp_nandrydays = xr.where(var_temp < 1.0, np.nan, var_temp)
    # obs_temp_95th = obs_temp_nandrydays.reduce(np.nanpercentile, dim='time', q=95)
    # obs_temp_95th.attrs['units'] = "mm/d"
    # R95p = xclim.indicators.icclim.R95p(var_temp, obs_temp_95th, freq='AS')

    # create datset to store variables
    # dict_out_temp = xr.Dataset(coords = R95p.coords)
    # dict_out_temp['R95p'] = R95p

    # RX5day
    dict_out_temp['RX5day'] = xclim.indicators.icclim.RX5day(var_temp, freq='AS').fillna(fillna)
    # SDII
    dict_out_temp['SDII'] = xclim.indicators.icclim.SDII(var_temp, freq='AS').fillna(fillna)
    # CDD
    # dict_out_temp['CDD'] = xclim.indicators.icclim.CDD(var_temp, freq='AS').fillna(fillna)
    # CWD
    # dict_out_temp['CWD'] = xclim.indicators.icclim.CWD(var_temp, freq='AS').fillna(fillna)

    first_item = True
    for ikey in dict_out_temp:
        if (first_item):
            ds_out_temp = xr.Dataset(coords = dict_out_temp[ikey].coords)
            first_item = False
        ds_out_temp[ikey] = dict_out_temp[ikey]
        
    return ds_out_temp

In [ ]:
for mod_name in ['obs']:
# for mod_name in var:
    print("dealing " + mod_name)
    mod_time  = var[mod_name]
    var_4to5 = var[mod_name].isel({"time":mod_time.time.dt.month.isin([4,5])})
    var_6to8 = var[mod_name].isel({"time":mod_time.time.dt.month.isin([6,7,8])})

    # ds_out_temp_4to5 = calc_extreme_pre(var_4to5, -1e30)
    # ds_out_temp_6to8 = calc_extreme_pre(var_6to8, -1e30)
    
    # comp = {'_FillValue' : -1e30}
    # encoding = {var:comp for var in list(ds_out_temp_4to5.data_vars)}

    # ds_out_temp_4to5.to_netcdf("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/extreme/extreme_"+mod_name+"_4-5.nc", encoding = encoding)
    # ds_out_temp_6to8.to_netcdf("/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/extreme/extreme_"+mod_name+"_6-8.nc", encoding = encoding)

    # del(ds_out_temp_4to5)
    # del(ds_out_temp_6to8)

In [ ]:
var_4to5

In [ ]:
var_temp = var_4to5

In [ ]:
var_temp = var_4to5
var_temp = var_temp.rename("pr")
var_temp.attrs['units'] = "mm/d"
fillna = -1e30
dict_out_temp = {}
dict_out_temp['RX5day_AM'] = xclim.indicators.icclim.RX5day(var_temp[0:122], freq='YS')

In [ ]:
lat = var_temp.lat
lon = var_temp.lon
allyeartime = pd.date_range('1998-01-01', '2017-12-31', freq = 'D')
allyeartime = xr.DataArray(allyeartime, name = 'time', coords = {"time":allyeartime}, dims = 'time')
allyear3dvar = xr.DataArray(name = 'empty3dvar', coords = {"time":allyeartime, "lat":lat, "lon":lon}, dims = ['time','lat','lon'])

In [ ]:
def expand_timecoords(da, new_time_coords):
    """
用来给xclim的输出结果进行前处理的部分，扩展3D变量的时间维，以免错误识别了NAN，将一年中不完整的部分填充为NAN
da dims: time lat lon
    """
    lat = da.lat
    lon = da.lon
    allyeartime = new_time_coords
    da_newtime  = xr.DataArray(name = 'empty3dvar', coords = {"time":allyeartime, "lat":lat, "lon":lon}, dims = ['time','lat','lon'])
    da_newtime.loc[{"time":da.time}] = da
    return da_newtime

In [ ]:
var_4to5_expand = expand_timecoords(var_4to5, allyeartime)

In [ ]:
var_4to5_expand[91].plot.contourf(cmap = 'gist_ncar', levels = np.linspace(0,20,21))

In [ ]:
allyear3dvar_temp = allyear3dvar.copy)

In [ ]:
time_sel = var_4to5.time

In [ ]:
allyear3dvar_temp.loc[{"time":time_sel}] = var_4to5

In [ ]:
var_temp = allyear3dvar_temp
var_temp = var_temp.rename("pr")
var_temp.attrs['units'] = "mm/d"
fillna = -1e30
dict_out_temp = {}
dict_out_temp['RX5day_AM'] = xclim.indicators.icclim.RX5day(var_temp, freq='YS')

In [ ]:
dict_out_temp['RX5day_AM'].time#[17].max()
dict_out_temp['RX5day_AM'][4].plo

In [ ]:
dict_out_temp['RX5day_AM'].mean(dim = 'time').plot.contourf(levels = np.linspace(0,200,11), cmap = 'gist_ncar')